### ONLY FOR TESTING PURPOSES, CANNOT BE USED IN PRODUCTION
##### This is a Kaggle Jupyter Notebook, therefore some restrictions apply when running Airflow in this environment. For example, the scheduling functions are restricted => we tested the execution of the DAG one time instead of scheduling it.

##### This script connects to the API Football data to fetch the Premier League standings results in a json file (Extract). Then, the useful data is arranged into a list of tuples (Transform). Finally, the script connects to Azure Database for MySQL Server to update a table with the standings results (Load).


In [25]:
# install mysql-connector-python package to connect with MySQL
!pip install mysql-connector-python 



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 47.7 MB/s eta 0:00:00:00:0100:01


In [11]:
# Orchestration - install apache Airflow

!pip install apache-airflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 36.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 53.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [12]:
# create airflow folder
import os
os.environ["AIRFLOW_HOME"] = "/kaggle/working/airflow"

In [13]:
# update database
!airflow db migrate

[2025-09-17T01:55:20.533+0000] {providers_manager.py:953} INFO - The hook_class 'airflow.providers.standard.hooks.filesystem.FSHook' is not fully initialized (UI widgets will be missing), because the 'flask_appbuilder' package is not installed, however it is not required for Airflow components to work
[2025-09-17T01:55:20.535+0000] {providers_manager.py:953} INFO - The hook_class 'airflow.providers.standard.hooks.package_index.PackageIndexHook' is not fully initialized (UI widgets will be missing), because the 'flask_appbuilder' package is not installed, however it is not required for Airflow components to work
DB: sqlite:////kaggle/working/airflow/airflow.db
Performing upgrade to the metadata database sqlite:////kaggle/working/airflow/airflow.db
[2025-09-17T01:55:20.759+0000] {migration.py:211} INFO - Context impl SQLiteImpl.
[2025-09-17T01:55:20.760+0000] {migration.py:214} INFO - Will assume non-transactional DDL.
[2025-09-17T01:55:20.761+0000] {migration.py:211} INFO - Context impl

**Create standings_dag.py necessary for orchestration with Airflow---------------------------------------------------------------**

In [79]:
%%writefile standings_dag.py
from airflow import DAG
from airflow.providers.standard.operators.python import PythonOperator
from datetime import timedelta, datetime 
import pandas as pd 
import json
import requests
import mysql.connector
from kaggle_secrets import UserSecretsClient # enable kaggle secrets for this notebook to replace .env file


# retrieve credentials to connect API
API_KEY = UserSecretsClient().get_secret("API_KEY")
API_HOST = UserSecretsClient().get_secret("API_HOST")
url = API_HOST
headers = {"X-Auth-Token": API_KEY}

# retrieve credentials to connect to MySQL Server
MYSQL_HOST = UserSecretsClient().get_secret("MYSQL_HOST")
MYSQL_PORT = UserSecretsClient().get_secret("MYSQL_PORT")
MYSQL_USER = UserSecretsClient().get_secret("MYSQL_USER")
MYSQL_PWD = UserSecretsClient().get_secret("MYSQL_PWD")
MYSQL_DB = UserSecretsClient().get_secret("MYSQL_DB")

# define extract_data() function
def extract_data():
    print("fetching data from Football Data API")
    try:
        response = requests.get(url, headers = headers)
        response.raise_for_status() # raises an exception for http errors
        print("API response received sucessfully")
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"An error occured: {e}")
        raise

# define transform_data() function
# parse clubs records into a list of tuples
def transform_data():
    data = extract_data()
    standings_list = data["standings"][0]["table"] # filter data down to the standings table list
    rows = [] 
    for club in standings_list:
        season = 2025
        position = club["position"]
        team_id = club["team"]["id"]
        team = club["team"]["name"]
        played = club["playedGames"]
        won = club["won"]
        draw = club["draw"]
        lost = club["lost"]
        goals_for = club["goalsFor"]
        goals_against = club["goalsAgainst"]
        goal_diff = club["goalDifference"]
        points = club["points"]
        form = club["form"]
        club_records = (season, position, team_id, team, played, won, draw, lost, goals_for, goals_against, goal_diff, points, form)
        rows.append(club_records)
    return rows

# define connect_to_db() function
def connect_to_db():
    print("connect to MySQL database")
    try:
        conn = mysql.connector.connect(
            host = MYSQL_HOST,
            port = MYSQL_PORT,
            user = MYSQL_USER,
            password = MYSQL_PWD,
            database = MYSQL_DB,
            ssl_ca = "/kaggle/input/ca-cert/DigiCertGlobalRootG2.crt.pem", ssl_disabled = False,
            connection_timeout = 10, # process will stops after 10asec without a response
            autocommit = False, # transactions are written in the database ONLY when we call a commit operation
            raise_on_warnings = True # turns MySQL errors into python exceptions
        )
        if conn.is_connected():
           print(f"Connection to database successful!") 
        return conn
    except Error as e:
        print(f"Database connection failed: {e}")
        raise

# define create_table() function
def create_table():
    conn = connect_to_db()
    print("Create table if not exist")
    try:
        cursor = conn.cursor()
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS standings (
        season INT NOT NULL,
        position INT NOT NULL,
        team_id INT NOT NULL,
        team VARCHAR(100) NOT NULL,
        played INT NOT NULL,
        won INT NOT NULL,
        draw INT NOT NULL,
        lost INT NOT NULL,
        goals_for INT NOT NULL,
        goals_against INT NOT NULL,
        goal_diff INT NOT NULL,
        points INT NOT NULL,
        form VARCHAR(5),
        PRIMARY KEY (season, team_id),
        UNIQUE KEY unique_season_pos(season, position)
        );
        """)
        conn.commit()
        print("Table was created")
    except Exception as e:
        print(f"Failed to create table: {e}")

# define upsert_records() function to update the standings table in MySQL database
def upsert_records():
    UPSERT_SQL = """INSERT INTO standings (season, position, team_id, team, played, won, draw, lost, goals_for, goals_against, goal_diff, points, form)
    VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s) AS src
    ON DUPLICATE KEY UPDATE
    position = src.position,
    team = src.team,
    played = src.played,
    won = src.won,
    draw = src.draw,
    lost = src.lost,
    goals_for = src.goals_for,
    goals_against = src.goals_against,
    goal_diff = src.goal_diff,
    points = src.points,
    form = src.form"""
    
    data = transform_data()
    conn = connect_to_db()
    print("Insert and update standings results into the database")
    no_rows = len(data) # number of rows in the table
    cursor = conn.cursor()
    try:
        cursor.executemany(UPSERT_SQL, data)
        conn.commit()
        print(f"[SUCCESS] - Upsert completed for {no_rows} rows")
    except Exception as e:
        conn.rollback()
        print(f"[ERROR] - Rolled back due to the following error: {e}")
    finally:
        cursor.close()
        conn.close()
        print("\nAll database connections closed. \nClean up completed.")

# DAG settings
defaults_args= {
    "owner": "jnh",
    "start_date": datetime(2025, 9, 16),
    "depends_on_past": False,
    "email_on_failure": False,
    "email_on_retry": False,
    "retries": 1,
    "retries_delay": timedelta(minutes = 5)
}
with DAG("standings_dag",
        default_args = defaults_args,
        catchup = False) as dag:
        new_table = PythonOperator(
        task_id = "new_table",
        python_callable = create_table
        )
        load = PythonOperator(
        task_id = "load",
        python_callable = upsert_records
        )
        new_table>>load

Overwriting standings_dag.py


In [19]:
# create dags folder to host standings_dag.py
folder = "dags"
folder_path = os.path.join("/kaggle/working/airflow", folder)
os.makedirs(folder_path, exist_ok = True)

In [80]:
import shutil
# delete standings_dags in dags folder if exists
if os.path.exists("/kaggle/working/airflow/dags/standings_dag.py"):
    os.remove("/kaggle/working/airflow/dags/standings_dag.py")
    print("file deleted")

# move standings_dag.py in dags folder
shutil.move("/kaggle/working/standings_dag.py", "/kaggle/working/airflow/dags/standings_dag.py")

file deleted


'/kaggle/working/airflow/dags/standings_dag.py'

In [81]:
# test airflow DAG execution
!airflow dags test standings_dag

[2025-09-17T04:11:18.228+0000] {providers_manager.py:953} INFO - The hook_class 'airflow.providers.standard.hooks.filesystem.FSHook' is not fully initialized (UI widgets will be missing), because the 'flask_appbuilder' package is not installed, however it is not required for Airflow components to work
[2025-09-17T04:11:18.229+0000] {providers_manager.py:953} INFO - The hook_class 'airflow.providers.standard.hooks.package_index.PackageIndexHook' is not fully initialized (UI widgets will be missing), because the 'flask_appbuilder' package is not installed, however it is not required for Airflow components to work
[2025-09-17T04:11:18.289+0000] {manager.py:122} INFO - DAG bundles loaded: dags-folder, example_dags
[2025-09-17T04:11:18.290+0000] {dagbag.py:585} INFO - Filling up the DagBag from /kaggle/working/airflow/dags
[2025-09-17T04:11:18.529+0000] {utils.py:164} INFO - NumExpr defaulting to 4 threads.
Nothing to clear.
[2025-09-17T04:11:19.598+0000] {dag.py:2306} INFO - created dagrun